<a href="https://colab.research.google.com/github/wndgua0468/machine_learning/blob/main/%EA%B8%B0%EA%B3%84%ED%95%A9%EC%8A%B5_14%EC%A3%BC%EC%B0%A8_%EC%8B%9C%EA%B3%84%EC%97%B4_%EB%8D%B0%EC%9D%B4%ED%84%B0%2C_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
# TIME SERIES QUESTION 시계열 데이터, LSTM
#
# Build and train a neural network to predict the time indexed variable of
# the univariate US diesel prices (On - Highway) All types for the period of
# 1994 - 2021.
# Using a window of past 10 observations of 1 feature , train the model
# to predict the next 10 observations of that feature.
#
# ==============================================================================

# =========== 합격 기준 가이드라인 공유 ============= #
# =================================================== #
# 문제명: Category 5 - Weekly US Retail Price
# val_mae: 0.026 혹은 그 이하
# =================================================== #
# =================================================== #

import urllib
import os
import zipfile
import pandas as pd
import tensorflow as tf



from tensorflow.keras.layers import Dense, Conv1D, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import Huber

# This function normalizes the dataset using min max scaling.
# DO NOT CHANGE THIS CODE
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data

# DO NOT CHANGE THIS.
def windowed_dataset(series, batch_size, n_past=10, n_future=10, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size=n_past + n_future, shift=shift, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(n_past + n_future))
    ds = ds.map(lambda w: (w[:n_past], w[n_past:]))
    return ds.batch(batch_size).prefetch(1)


def solution_model():
    # DO NOT CHANGE THIS CODE
    # Reads the dataset.
    df = pd.read_csv('Weekly_U.S.Diesel_Retail_Prices.csv',
                     infer_datetime_format=True, index_col='Week of', header=0)
    
    N_FEATURES = len(df.columns) # DO NOT CHANGE THIS
    
    # Normalizes the data
    data = df.values
    data = normalize_series(data, data.min(axis=0), data.max(axis=0))
    
    # Splits the data into training and validation sets.
    SPLIT_TIME = int(len(data) * 0.8) # DO NOT CHANGE THIS
    x_train = data[:SPLIT_TIME]
    x_valid = data[SPLIT_TIME:]

    # DO NOT CHANGE THIS CODE
    tf.keras.backend.clear_session()
    tf.random.set_seed(42)

    BATCH_SIZE = 32  # ADVISED NOT TO CHANGE THIS
    N_PAST = 10  # DO NOT CHANGE THIS
    N_FUTURE = 10  # DO NOT CHANGE THIS
    SHIFT = 1  # DO NOT CHANGE THIS

    train_set = windowed_dataset(series=x_train, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)
    valid_set = windowed_dataset(series=x_valid, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)
    # Code to define your model.
    model = tf.keras.models.Sequential([
        Conv1D(filters=32, kernel_size=5, padding='causal', activation='relu', input_shape=[N_PAST, 1]),
        Bidirectional(LSTM(32, return_sequences=True)),
        Bidirectional(LSTM(32, return_sequences=True)),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(N_FEATURES)
    ])
    checkpoint_path='model/my_checkpoint.ckpt'

    checkpoint = ModelCheckpoint(checkpoint_path,
                             save_weights_only=True,
                             save_best_only=True,
                             monitor='val_loss',
                             verbose=1,
                             )
    # Code to train and compile the model
    optimizer =  tf.keras.optimizers.Adam(0.00003)  # YOUR CODE HERE
    model.compile(optimizer=optimizer, loss=tf.keras.losses.Huber(), metrics=['mae'])

    model.fit(train_set, validation_data=(valid_set), epochs=150, callbacks=[checkpoint])

    model.load_weights(checkpoint_path)


    return model

if __name__ == '__main__':
    model = solution_model()
    model.save("model5.h5")

In [ ]:
##데이터 확인하기

import urllib
import pandas as pd
import tensorflow as tf

In [ ]:
url = "https://www.dropbox.com/s/eduk281didil1km/Weekly_U.S.Diesel_Retail_Prices.csv?dl=1"
urllib.request.urlretrieve(url, 'Weekly_U.S.Diesel_Retail_Prices.csv')

In [ ]:
pd.read_csv("Weekly_U.S.Diesel_Retail_Prices.csv", infer_datetime_format=True,
            index_col='Week of', header=0)